In [2]:
import transformers
from transformers import ElectraModel, ElectraTokenizer
import torch
import pandas as pd
import torch.nn as nn
import random
import os
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
# 코랩에서 실행
# from google.colab import drive
# drive.mount('content/gdrive')

In [4]:
# !pip install Korpora
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

# %cd Mecab-ko-for-Google-Colab/

# !bash install_mecab-ko_on_colab_light_220429.sh

# from konlpy.tag import Mecab

# tokenizer_mecab = Mecab()

In [4]:
# from konlpy.tag import Komoran

# komoran = Komoran()

In [3]:
electra_version = 'base-v3' # can be base / small / base-v2 / small-v2 / base-v3 / small-v3 바꾸면 맨 위에 tokenizer도 바꿔줘야함
tokenizer = ElectraTokenizer.from_pretrained(f"monologg/koelectra-{electra_version}-discriminator")

In [6]:
# from google.colab import files
# import chardet

# with open('kodoli_v1.1.csv', 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(10000))

# # check what the character encoding might be
# print(result)

In [4]:
# 데이터 받아서 문장 앞 뒤에 class 토큰 seperate 토큰 넣어주기
df = pd.read_csv('kodoli_v1.1.csv', encoding='CP949')
data_size = len(df)
data_list = []
for i in range(data_size):
  data_list.append(list(df.iloc[i, 1:5]))
for i in range(data_size):
  data_list[i][0] = '[CLS] ' + data_list[i][0] + ' [SEP]'

In [4]:
# 문장 토큰 개수 분포
len_list = [0 for i in range(250)]
for data in data_list:
  lenOfSen = len(tokenizer.tokenize(data[0]))
  len_list[lenOfSen] += 1
sum = 0
for i in range(1, 250):
  sum += len_list[i]
  print(f'N of Seq with len{i} : {len_list[i]} / cumulative rate: {sum / data_size *100}%')

N of Seq with len1 : 0 / cumulative rate: 0.0%
N of Seq with len2 : 0 / cumulative rate: 0.0%
N of Seq with len3 : 81 / cumulative rate: 0.21218074656188607%
N of Seq with len4 : 134 / cumulative rate: 0.5631958087753766%
N of Seq with len5 : 527 / cumulative rate: 1.9436804191224626%
N of Seq with len6 : 969 / cumulative rate: 4.4819908316961365%
N of Seq with len7 : 1227 / cumulative rate: 7.696136214800261%
N of Seq with len8 : 1634 / cumulative rate: 11.976424361493123%
N of Seq with len9 : 1931 / cumulative rate: 17.034708578912902%
N of Seq with len10 : 2080 / cumulative rate: 22.48330058939096%
N of Seq with len11 : 2150 / cumulative rate: 28.115258677144727%
N of Seq with len12 : 2119 / cumulative rate: 33.66601178781926%
N of Seq with len13 : 2033 / cumulative rate: 38.99148657498363%
N of Seq with len14 : 2007 / cumulative rate: 44.24885396201702%
N of Seq with len15 : 1873 / cumulative rate: 49.155206286836936%
N of Seq with len16 : 1708 / cumulative rate: 53.62933857236412%

In [5]:
# data의 약 95%를 수용하는 길이인 64로 설정해서 64보다 많은 토큰이 있는 data는 버리기
max_length = 64
# label들을 숫자로 바꿔주기
abuse_dict = {'NON': 0, 'ABS' : 1}
sentiment_dict = {'NEG': 0, 'NEU': 1, 'POS': 2}
offensive_dict = {'NOT': 0, 'LIKELY': 1, 'OFFEN': 2}
# 모든 문장을 토큰화하고 숫자로 바꾼다. max length 보다 작은 문장은 pad 토큰을 넣어준다.
sentence_ids_list = []
attn_masks_list = []
labels_list = []

for i in range(38175):
  temp = []
  tokened = tokenizer.tokenize(data_list[i][0])
  idx = tokenizer.convert_tokens_to_ids(tokened)
  idx_len = len(idx)
  attention_mask = [1 for i in range(idx_len)]
  if idx_len <= max_length:
    num_pad = max_length - idx_len
    for j in range(num_pad):
      idx.append(0)
      attention_mask.append(0)
  else: # max length 보다 긴 건 데이터셋에 안 넣기
    continue
  sentence_ids_list.append(idx)
  attn_masks_list.append(attention_mask)
  temp.append(abuse_dict[data_list[i][1]])
  temp.append(sentiment_dict[data_list[i][2]])
  temp.append(offensive_dict[data_list[i][3]])
  labels_list.append(temp)

In [6]:
# train / test 데이터셋 나누기

test_data_percent = 10

total_data_size = len(sentence_ids_list) #36,342
test_data_size = int(total_data_size * (test_data_percent / 100)) #3,634 10% test data
train_data_size = total_data_size - test_data_size #32,708 90% train data

train_sentence_ids_list = torch.tensor(sentence_ids_list[:train_data_size])
test_sentence_ids_list = torch.tensor(sentence_ids_list[train_data_size:])
train_attn_masks_list = torch.tensor(attn_masks_list[:train_data_size])
test_attn_masks_list = torch.tensor(attn_masks_list[train_data_size:])
train_labels_list = torch.tensor(labels_list[:train_data_size])
test_labels_list = torch.tensor(labels_list[train_data_size:])

In [7]:
batch_size = 64

train_data = torch.utils.data.TensorDataset(train_sentence_ids_list, train_attn_masks_list, train_labels_list)
train_sampler = torch.utils.data.RandomSampler(train_data)
train_dataloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = torch.utils.data.TensorDataset(test_sentence_ids_list, test_attn_masks_list, test_labels_list)
test_sampler = torch.utils.data.RandomSampler(test_data)
test_dataloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [10]:
class Classifier(nn.Module):
  def __init__(self, c_in, c_out):
    super().__init__()
    self.dropout = nn.Dropout(p = 0.5)
    self.fc1 = nn.Linear(c_in, c_out)
    self.norm = nn.LayerNorm(c_out)
    self.activation = nn.ReLU()
    self.init_weights()

  def forward(self, x):
    x = self.dropout(x)
    return self.activation(self.norm(self.fc1(x)))

  def init_weights(self):
    torch.nn.init.kaiming_normal_(self.fc1.weight.data)
    torch.nn.init.zeros_(self.fc1.bias.data)

In [11]:
class NLPteam2_KoElectra(nn.Module):
    def __init__(self, electra_version = 'base-v3', num_hiddens = 256):
        super().__init__()
        self.electra = ElectraModel.from_pretrained(f"monologg/koelectra-{electra_version}-discriminator")
        electra_out_dim = 768 if 'base' in electra_version else 256
        
        self.abu_c1 = Classifier(electra_out_dim, 2)
        self.senti_c1 = Classifier(electra_out_dim, 3)
        self.off_c1 = Classifier(electra_out_dim, 3)

        for param in list(self.electra.named_parameters()):
            if '10' in param[0]:
                break
            param[1].requires_grad = False

    def forward(self, x, attention_mask):
        out = self.electra(x, attention_mask = attention_mask)
        cls_representation = out[0][:, 0, :]
        #cls_representation = torch.mean(out[0], dim=1)
        
        tmp_a = self.abu_c1(cls_representation)
        tmp_s = self.senti_c1(cls_representation)
        tmp_o = self.off_c1(cls_representation)
        
        # abu_pred = self.abu_c3(self.abu_c2(tmp_a))
        # senti_pred = self.senti_c3(self.senti_c2(tmp_s))
        # off_pred = self.off_c3(self.off_c2(tmp_o))

        # abu_pred = self.abu_c2(tmp_a)
        # senti_pred = self.senti_c2(tmp_s)
        # off_pred = self.off_c2(tmp_o)       
        
        # return (abu_pred, senti_pred, off_pred)
        return (tmp_a, tmp_s, tmp_o)



In [10]:
# # train / test 데이터셋 나누기

# test_data_percent = 20

# total_data_size = len(token_data_list) #36342
# test_data_size = int(total_data_size * (test_data_percent / 100)) #7268 20% test data
# train_data_size = total_data_size - test_data_size #29074 80% train data
# train_data = token_data_list[:train_data_size]
# test_data = token_data_list[train_data_size:]

In [24]:
#torch.manual_seed(42)
max_epochs = 20
max_step_per_epoch = None
lr = 1e-3
num_hiddens = 512 # classifier hidden dim
electra_version = 'base-v3' # can be base / small / base-v2 / small-v2 / base-v3 / small-v3
saved_model_name = None # 불러올 모델 저장한 파일명
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = NLPteam2_KoElectra(electra_version=electra_version, num_hiddens=num_hiddens)
if saved_model_name:
    model.load_state_dict(torch.load(os.getcwd() + f'/{saved_model_name}.pth'))    
model.to(device)

warmup_ratio = 0.1
t_total = len(train_dataloader) * max_epochs
warmup_step = int(t_total * warmup_ratio)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)
criterion = nn.CrossEntropyLoss()

In [13]:
model_param_name = []
for param in list(model.named_parameters()):
    if param[1].requires_grad:
        model_param_name.append(param[0])
model_param_name

['electra.encoder.layer.10.attention.self.query.weight',
 'electra.encoder.layer.10.attention.self.query.bias',
 'electra.encoder.layer.10.attention.self.key.weight',
 'electra.encoder.layer.10.attention.self.key.bias',
 'electra.encoder.layer.10.attention.self.value.weight',
 'electra.encoder.layer.10.attention.self.value.bias',
 'electra.encoder.layer.10.attention.output.dense.weight',
 'electra.encoder.layer.10.attention.output.dense.bias',
 'electra.encoder.layer.10.attention.output.LayerNorm.weight',
 'electra.encoder.layer.10.attention.output.LayerNorm.bias',
 'electra.encoder.layer.10.intermediate.dense.weight',
 'electra.encoder.layer.10.intermediate.dense.bias',
 'electra.encoder.layer.10.output.dense.weight',
 'electra.encoder.layer.10.output.dense.bias',
 'electra.encoder.layer.10.output.LayerNorm.weight',
 'electra.encoder.layer.10.output.LayerNorm.bias',
 'electra.encoder.layer.11.attention.self.query.weight',
 'electra.encoder.layer.11.attention.self.query.bias',
 'electr

In [25]:
trial = 79

In [26]:
# Pre-train Evaluation
abu_acc, senti_acc, off_acc, total_count = 0, 0, 0, 0

print(f'before {trial}th trial\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # gpu가속을 위함
model = model.to(device)
model.eval()

for data in test_dataloader:
  input = data[0].to(device)
  attn_mask = data[1].to(device)
  abu_true = data[2][:, 0]
  senti_true = data[2][:, 1]
  off_true = data[2][:, 2]

  out = model(input, attention_mask = attn_mask)
  abu_pred = out[0]
  senti_pred = out[1]
  off_pred = out[2]  


  abu_pred = abu_pred.detach().cpu()
  senti_pred = senti_pred.detach().cpu()
  off_pred = off_pred.detach().cpu()

  abu_acc += (abu_pred.argmax(1) == abu_true).sum().item()
  senti_acc += (senti_pred.argmax(1) == senti_true).sum().item()
  off_acc += (off_pred.argmax(1) == off_true).sum().item()

  total_count += abu_pred.size(0)

  # if total_count % (batch_size * 20) == 0:
  #   print(f'평가 진행중.. [{total_count}/{test_data_size}]')
  #   print(f"학습된 모델의 중간 abussive detection 정확도: {format(abu_acc/total_count * 100, '.3f')} %")
  #   print(f"학습된 모델의 중간 sentiment analysis 정확도: {format(senti_acc/total_count * 100, '.3f')} %")
  #   print(f"학습된 모델의 중간 offensive language detection 정확도: {format(off_acc/total_count * 100, '.3f')} %")


print(f"초기화된 모델의 ALD 정확도: {format(abu_acc/total_count * 100, '.2f')} %")
print(f"초기화된 모델의 SA 정확도: {format(senti_acc/total_count * 100, '.2f')} %")
print(f"초기화된 모델의 OLI 정확도: {format(off_acc/total_count * 100, '.2f')} %")

before 79th trial

초기화된 모델의 ALD 정확도: 56.05 %
초기화된 모델의 SA 정확도: 34.73 %
초기화된 모델의 OLI 정확도: 13.95 %


In [27]:
#max_epochs = 10
eval_per_epoch = 2
# training
model.train()
abu_acc, senti_acc, off_acc, total_count = 0, 0, 0, 0
for epoch in range(max_epochs):
  num_steps = 0
  
  for batch in train_dataloader:
    input = batch[0].to(device)
    attn_mask = batch[1].to(device)
    abu_true = batch[2][:, 0].to(device)
    senti_true = batch[2][:, 1].to(device)
    off_true = batch[2][:, 2].to(device)

    out = model(input, attention_mask = attn_mask)

    abu_pred = out[0]
    senti_pred = out[1]
    off_pred = out[2]  


    abu_loss = criterion(abu_pred, abu_true)
    senti_loss = criterion(senti_pred, senti_true)
    off_loss = criterion(off_pred, off_true)
    
    loss = abu_loss + senti_loss + off_loss

    #loss = criterion(abu_pred, abu_true)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step
    num_steps += batch_size
    if (num_steps) % (batch_size * 100) == 0:
      print(f'Epoch {epoch+1}/{max_epochs}, Step [{num_steps}/{train_data_size}], Loss: {loss.item():.4f}')

    abu_acc += (abu_pred.argmax(1) == abu_true).sum().item()
    senti_acc += (senti_pred.argmax(1) == senti_true).sum().item()
    off_acc += (off_pred.argmax(1) == off_true).sum().item()
    total_count += abu_pred.size(0)

    if max_step_per_epoch and num_steps > max_step_per_epoch:
      break
    
  
  if epoch % eval_per_epoch == eval_per_epoch-1:
    print(f"모델의 ALD 정확도: {format(abu_acc/total_count * 100, '.2f')} %")
    print(f"모델의 SA 정확도: {format(senti_acc/total_count * 100, '.2f')} %")
    print(f"모델의 OLI 정확도: {format(off_acc/total_count * 100, '.2f')} %")
    abu_acc, senti_acc, off_acc, total_count = 0, 0, 0, 0
    

Epoch 1/20, Step [6400/32708], Loss: 3.4244
Epoch 1/20, Step [12800/32708], Loss: 3.3128
Epoch 1/20, Step [19200/32708], Loss: 3.6013
Epoch 1/20, Step [25600/32708], Loss: 3.2868
Epoch 1/20, Step [32000/32708], Loss: 3.4168
Epoch 2/20, Step [6400/32708], Loss: 3.2380
Epoch 2/20, Step [12800/32708], Loss: 3.2002
Epoch 2/20, Step [19200/32708], Loss: 3.2200
Epoch 2/20, Step [25600/32708], Loss: 3.4596
Epoch 2/20, Step [32000/32708], Loss: 3.4435
모델의 ALD 정확도: 51.63 %
모델의 SA 정확도: 32.67 %
모델의 OLI 정확도: 26.18 %
Epoch 3/20, Step [6400/32708], Loss: 3.4212
Epoch 3/20, Step [12800/32708], Loss: 3.3966
Epoch 3/20, Step [19200/32708], Loss: 3.3713
Epoch 3/20, Step [25600/32708], Loss: 3.3476
Epoch 3/20, Step [32000/32708], Loss: 3.2497
Epoch 4/20, Step [6400/32708], Loss: 3.3992
Epoch 4/20, Step [12800/32708], Loss: 3.2838
Epoch 4/20, Step [19200/32708], Loss: 3.6909
Epoch 4/20, Step [25600/32708], Loss: 3.2659
Epoch 4/20, Step [32000/32708], Loss: 3.3852
모델의 ALD 정확도: 51.35 %
모델의 SA 정확도: 32.84 %
모

In [28]:
# Post-train Evaluation
abu_acc, senti_acc, off_acc, total_count = 0, 0, 0, 0

print(f'after {trial}th trial\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # gpu가속을 위함
model = model.to(device)
model.eval()

for data in test_dataloader:
  input = data[0].to(device)
  attn_mask = data[1].to(device)
  abu_true = data[2][:, 0]
  senti_true = data[2][:, 1]
  off_true = data[2][:, 2]

  out = model(input, attention_mask = attn_mask)
  abu_pred = out[0]
  senti_pred = out[1]
  off_pred = out[2]  

  
  abu_pred = abu_pred.detach().cpu()
  senti_pred = senti_pred.detach().cpu()
  off_pred = off_pred.detach().cpu()

  abu_acc += (abu_pred.argmax(1) == abu_true).sum().item()
  senti_acc += (senti_pred.argmax(1) == senti_true).sum().item()
  off_acc += (off_pred.argmax(1) == off_true).sum().item()

  total_count += abu_pred.size(0)

  # if total_count % (batch_size * 20) == 0:
  #   print(f'평가 진행중.. [{total_count}/{test_data_size}]')
  #   print(f"학습된 모델의 중간 abussive detection 정확도: {format(abu_acc/total_count * 100, '.3f')} %")
  #   print(f"학습된 모델의 중간 sentiment analysis 정확도: {format(senti_acc/total_count * 100, '.3f')} %")
  #   print(f"학습된 모델의 중간 offensive language detection 정확도: {format(off_acc/total_count * 100, '.3f')} %")


print(f"학습된 모델의 최종 ALD 정확도: {format(abu_acc/total_count * 100, '.2f')} %")
print(f"학습된 모델의 최종 SA 정확도: {format(senti_acc/total_count * 100, '.2f')} %")
print(f"학습된 모델의 최종 OLI 정확도: {format(off_acc/total_count * 100, '.2f')} %")

after 79th trial

학습된 모델의 최종 ALD 정확도: 56.05 %
학습된 모델의 최종 SA 정확도: 34.73 %
학습된 모델의 최종 OLI 정확도: 13.95 %


In [ ]:
# Post-train thorough Evaluation
TPFP_not, TP_not, TPFN_not = 0, 0, 0
TPFP_likely, TP_likely, TPFN_likely = 0, 0, 0
TPFP_off, TP_off, TPFN_off = 0, 0, 0
total_count = 0

print(f'after {trial}th trial / offensive detection thorough evaluation\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # gpu가속을 위함
model = model.to(device)
model.eval()

for data in test_dataloader:
  input = data[0].to(device)
  attn_mask = data[1].to(device)
  off_true = data[2][:, 2]

  out = model(input, attention_mask = attn_mask)

  out = out.detach().cpu()
  predict_index = out.argmax(1)

  off_pred = (predict_index % 9) % 3
    
  TPFP_not += (off_pred == torch.tensor(0)).sum().item()
  TPFP_likely += (off_pred == torch.tensor(1)).sum().item()
  TPFP_off += (off_pred == torch.tensor(2)).sum().item()

  TP_not += (off_pred * 10 == off_true).sum().item()
  TP_likely += (((off_pred - torch.tensor(1))*10 + torch.tensor(1)) == off_true).sum().item()
  TP_off += (((off_pred - torch.tensor(2)) *10 + torch.tensor(2)) == off_true).sum().item()

  TPFN_not += (off_true == torch.tensor(0)).sum().item()
  TPFN_likely += (off_true == torch.tensor(1)).sum().item()
  TPFN_off += (off_true == torch.tensor(2)).sum().item()

  total_count += out.size(0)

print(TPFP_not, TP_not, TPFN_not,  '/', TPFP_likely, TP_likely, TPFN_likely, '/', TPFP_off, TP_off, TPFN_off, '/', total_count)
print(f"학습된 모델의 Not 정확도: Precision {format(TP_not/TPFP_not * 100, '.2f')} % / Recall {format(TP_not/TPFN_not * 100, '.2f')} % / F1 {format((2 * TP_not/TPFP_not * 100 * TP_not/TPFN_not * 100) / (TP_not/TPFP_not * 100 + TP_not/TPFN_not * 100), '.2f')}")
print(f"학습된 모델의 Likely 정확도: Precision {format(TP_likely/TPFP_likely * 100, '.2f')} % / Recall {format(TP_likely/TPFN_likely * 100, '.2f')} % / F1 {format((2 * TP_likely/TPFP_likely * 100 * TP_likely/TPFN_likely * 100) / (TP_likely/TPFP_likely * 100 + TP_likely/TPFN_likely * 100), '.2f')}")
print(f"학습된 모델의 Offensive 정확도: Precision {format(TP_off/TPFP_off * 100, '.2f')} % / Recall {format(TP_off/TPFN_off * 100, '.2f')} % / F1 {format((2 * TP_off/TPFP_off * 100 * TP_off/TPFN_off * 100) / (TP_off/TPFP_off * 100 + TP_off/TPFN_off * 100), '.2f')}")

In [44]:
# # 결과 잘 나온 모델 저장하기
# savefile_name = f'{trial}th_trial'
# Path = os.getcwd() # <- 로컬
# #Path = '/content/gdrive/MyDrive/Colab Notebooks'# <- 코랩
# torch.save(model.state_dict(), Path + f'/{savefile_name}.pth')